# Install soamc-python-sqs-client:

In [ ]:
%%bash

# need the following line to install soamc_submitter.py as a script
#pip install git+https://github.jpl.nasa.gov/SOAMC/soamc-python-sqs-client.git

# for now, clone it instead
cd /tmp
git clone https://github.jpl.nasa.gov/SOAMC/soamc-python-sqs-client.git
cd soamc-python-sqs-client
pip install .

# Create the `sqsconfig.py` configuration file so that you can communicate with the WPST listener in EKS:

In [ ]:
import getpass

# prompt for AWS account ID
aws_account_id = input("Enter AWS account ID then press <Enter>: ")

# prompt for AWS access key
print("Enter AWS access key then press <Enter>.")
aws_access_key = getpass.getpass()

# prompt for AWS secret key
print("Enter AWS secret key then press <Enter>.")
aws_secret_key = getpass.getpass()

# prompt for AWS region
aws_region = input("Enter AWS region then press <Enter>: ")

# prompt for SQS name
listener_queue = input("Enter SQS queue name then press <Enter>: ")

# create the config file
with open("sqsconfig.py", "w") as f:
    f.write(f"""[AWS_SQS_QUEUE]
AWS_ACCOUNT_ID={aws_account_id}
region_name={aws_region}
aws_access_key = {aws_access_key}
aws_secret_key = {aws_secret_key}
listener_queue = {listener_queue}
poll_interval = 60
queue_visibility_timeout = 600
error_queue_name = None
error_queue_visibility_timeout = 600
reply_timeout_sec = 20
execute_reply_timeout_sec = 600
force_delete = False
wait_time = 0
max_number_of_messages =  1
fifo_group_id = SOAMC_DEFAULT_GROUP1
queue_url = https://sqs.{aws_region}.amazonaws.com/{aws_account_id}/{listener_queue}

[DAEMON]
PID_FILE_PATH=/home/jovyan/sqs_daemon.pid
DAEMON_OUTPUT_OVERWRITE=False 
DAEMON_OUTPUT_FILE=/home/jovyan/test_out
DAEMON_ERROR_FILE=/home/jovyan/error_out
DAEMON_STDIN=/dev/null

[ADES_WPS-T_SERVER]
wps_server_url=http://127.0.0.1:5000/
""")

# Deploy the `downsample-landsat` version 2.0.0 to the ADES: 

In [ ]:
%%bash

python /tmp/soamc-python-sqs-client/soamc_submitter.py \
  deployprocess \
  https://raw.githubusercontent.com/pymonger/downsample-landsat/2.0.0/workflow-application-descriptor.json

# Submit job

In [ ]:
import json

# workflow job parameters
job_params = {
    "workflow_input_url": "https://github.com/pymonger/downsample-landsat/releases/download/1.0.0/LC08_L1TP_065016_20130724_20170309_01_T1_BQA.TIF",
    "workflow_min_spin_time": 15,
    "workflow_max_spin_time": 30,
    "workflow_aws_access_key_id": None,
    "workflow_aws_secret_access_key": None,
    "workflow_base_dataset_url": "s3://hysds-dataset-bucket-gman-test/eks-via-ades"
}
with open("/tmp/workflow-inputs.json", "w") as f:
    json.dump(job_params, f, indent=2)

In [ ]:
%%bash

python /tmp/soamc-python-sqs-client/soamc_submitter.py \
  execute downsample-landsat-workflow-2.0.0 \
  /tmp/workflow-inputs.json 

# Monitor job

In [ ]:
# prompt for job id
job_id = input("Enter jobID and press <Enter>: ")

In [ ]:
%%bash -s "$job_id"

python /tmp/soamc-python-sqs-client/soamc_submitter.py \
  getstatus downsample-landsat-workflow-2.0.0 $1

# Get result

In [ ]:
%%bash -s "$job_id"

python /tmp/soamc-python-sqs-client/soamc_submitter.py \
  getresult downsample-landsat-workflow-2.0.0 $1